In [1]:
# Librerias
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats

In [7]:
quandl.ApiConfig.api_key = "Us7sh5qQJQtwrPCA3rc1" # Quandl API key\n"
fechas = list(['2017-03-31','2018-03-29'])

In [40]:
mes1 = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-1.csv', skiprows=10, delim_whitespace=False)
mes1=mes1.drop(columns = ['Name', 'Asset Class', "Market Value", 'Notional Value', 'Sector', 'SEDOL', 'ISIN', 'Exchange'])
mes1=mes1.dropna(axis=0)
#mes1

tick = list(mes1.index)
weights = mes1['Weight (%)']


In [5]:
t1 = mes1.iloc[:,0:1] #Tickers
peso1 = mes1.iloc[:,1] #Pesos 
precio1 = mes1.iloc[:,2] #Precio de la acción
a1 = peso1*precio1 # Valor de la acción por Ticker

v_etf1 = sum(a1) #Suma de los valores de las acciones
print ("El valor de todas las posiciones del ETF es de:", v_etf1)

ETF1 =  17300000/v_etf1
print("El valor del ETF es de", ETF1)


El valor de todas las posiciones del ETF es de: 12743.808399999996
El valor del ETF es de 1357.5219790655362


In [62]:
data_csv = pd.read_csv('https://raw.githubusercontent.com/Dianachav/Trading_Lab1/master/ITA_holdings-1.csv', skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'])
tick = list(data_csv.index)
weights = data_csv['Weight (%)']
def Download_QuandL(fechas:'Dates_list ', tick:'Tickers of Stock'):
    dates = { 'gte': fechas[0], 'lte': fechas[1] }
    column = { 'columns': ['date', 'Open', 'Close']}
    data = quandl.get_table('WIKI/PRICES', qopts = column, ticker = tick, date = dates, paginate=True)# ticker = 'WIKI/AAPL' #'AAPL.US'\n",
    data = data.iloc[::-1]
    data = data.reset_index()
    data = data.drop(['None'],1)
    return data

In [63]:
def All_Data(fechas:'Dates_list ', tick:'Ticker'):
    d = {} # Crea un Diccionario Vacío\n",
    for i in range(len(tick)): # ciclo que descarga los datos de Quandl y los guarda como DataFrames en un Diccionario\n",
        nom = str(tick[i]) 
        d["_{0}".format(nom)] = Download_QuandL(fechas, tick[i])
        return d


In [17]:
def Rebalanceo(df, Com, Cap_I, R_R, n_a, we, op):
    #Esta funcion hace los rebalanceos de cada activo de forma diaria por una año 
    a = (df['close']/df['open'])-1
    P_se = list(df['open']*.99)
    add_s = list()
    Opp = list()
    comi = list()
    In_shares = np.floor((we*Cap_I)/df['close'][0])
    for i in range(len(df)):
        if a.iloc[i] < R_R:
            add_s.append(n_a)
            Opp.append('Compra')
            comi.append(Com*df['close'].iloc[i]*n_a)
            
        else:
            add_s.append(0)
            Opp.append('-')
            comi.append(0)
    df = df.join(pd.DataFrame(add_s, columns = ['# Added Shares']))
    df = df.join(pd.DataFrame(Opp, columns = ['Operacion']))
    df = df.join(pd.DataFrame(comi, columns = ['Comisión']))
    df = df.join(pd.DataFrame(P_se, columns = ['Precio de Señal']))
    Tot_Acciones = pd.DataFrame(list(df['# Added Shares'].cumsum()), columns = ['Total Shares'])+In_shares
    df = df.join(Tot_Acciones)
    Val_to_port = pd.DataFrame(list(df['close']*df['Total Shares']), columns = ['Value to ETF'])
    df = df.join(Val_to_port)
    
    frames = [op, df[df.Operacion == 'Compra']]
    op = pd.concat(frames)
    
    
    return df, op
    

In [73]:
def Clean_Dicc(dic:'Diccionario a limpiar',sizes:'Tamaño deseado del DF'):
    y = list(dic.keys())
    for h in y:
        if len(dic[h])!=sizes:
            del dic[h]
            
    return dic

In [77]:
Data = All_Data(fechas, tick)

In [74]:
d_full = Clean_Dicc(Data,248)

In [52]:
tick = list(d_full.keys())
d2 = {}
op = pd.DataFrame(columns=['date', 'open', 'close', '# Added Shares', 'Operacion', 'Comisión', 'Precio de Señal', 'Total Shares', 'Value to ETF'])
for j in range(len(tick)):
    tabla , op = Rebalanceo(d_full[tick[j]],0.0003,1000000,-0.01,10,weights[j], op)
    d2["Data{0}".format(j)] = tabla
op_2 = op.groupby('date')

In [78]:
len(Data)

1